<table class="table table-bordered">
    <tr>
        <th style="text-align:center; vertical-align: middle; width:50%"><img src='https://www.np.edu.sg/images/default-source/default-album/img-logo.png'"></th>
        <th style="text-align:center;"><h1>Deep Learning</h1><h2>Assignment 1 - Food Classification Model (Individual)</h2><h3>AY2023/24 Semester</h3></th>
    </tr>
</table>

In [ ]:
!pip install keras-tuner

In [ ]:
# Import the Required Packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, LearningRateScheduler

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import keras_tuner as kt

import os

## Step 1: Data Loading

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
!ls "/content/drive/My Drive/"
base_dir = "/content/drive/My Drive/Ngee Ann Poly/Deep Learning/ASG1/Images"

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Ngee Ann Poly'  'Udemy Courses'


## Step 2:  Develop the Image Classification Models

In [ ]:
img_size = 224

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_size, img_size),
        batch_size=50,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(img_size, img_size),
        batch_size=50,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=50,
    class_mode='categorical')

Found 7500 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Found 500 images belonging to 10 classes.


In [ ]:
def build_model(hp):
    img_size = 224
    model = models.Sequential()
    model.add(layers.Resizing(img_size, img_size, crop_to_aspect_ratio=True, input_shape=(None, None, 3)))

    # Define hyperparameters to tune
    hp_filters = hp.Int('filters', min_value=32, max_value=512, step=32)
    hp_dense_units = hp.Int('units', min_value=1024, max_value=8192, step=1024)
    hp_conv_layers = hp.Int('conv_layers', min_value=2, max_value=5)
    hp_pooling = hp.Choice('pooling', values=['max', 'avg'])
    hp_kernel_size = hp.Choice('kernel_size', values=[3, 5, 7])  # Different kernel sizes to try

    # Add convolutional layers based on the hyperparameters
    for _ in range(hp_conv_layers):
        model.add(layers.Conv2D(hp_filters, (hp_kernel_size, hp_kernel_size), activation='relu', padding='same'))
        if hp_pooling == 'max':
            model.add(layers.MaxPooling2D((2, 2)))
        else:
            model.add(layers.AveragePooling2D((2, 2)))

    model.add(layers.GlobalAveragePooling2D())

    # Add dense layers based on the hyperparameter
    model.add(layers.Dense(hp_dense_units, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizers.Adam(learning_rate=0.0005),
        metrics=['acc']
    )

    return model

In [ ]:
checkpoint_filepath = '/content/drive/My Drive/Ngee Ann Poly/Deep Learning/ASG1/Saved Models/model_4.tf'

model_checkpoint = ModelCheckpoint(
    checkpoint_filepath,
    monitor='val_acc',
    save_best_only=True,
    mode='max',
    verbose=1
)

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=30,
    directory='/content/drive/My Drive/Ngee Ann Poly/Deep Learning/ASG1/Saved Models',
    project_name='Architecture Trials'
)

train_generator.reset()
validation_generator.reset()
test_generator.reset()

tuner.search(train_generator, validation_data=validation_generator, epochs=50, steps_per_epoch=150, validation_steps=40, callbacks=[model_checkpoint])


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
256               |256               |filters
6144              |6144              |units
3                 |3                 |conv_layers
avg               |avg               |pooling
5                 |5                 |kernel_size

Epoch 1/50
150/150 [==============================] - ETA: 0s - loss: 2.2491 - acc: 0.1387 
Epoch 1: val_acc improved from -inf to 0.16050, saving model to /content/drive/My Drive/Ngee Ann Poly/Deep Learning/ASG1/Saved Models/model_4.tf
150/150 [==============================] - 5467s 36s/step - loss: 2.2491 - acc: 0.1387 - val_loss: 2.2091 - val_acc: 0.1605
Epoch 2/50
150/150 [==============================] - ETA: 0s - loss: 2.2118 - acc: 0.1616 
Epoch 2: val_acc improved from 0.16050 to 0.16750, saving model to /content/drive/My Drive/Ngee Ann Poly/Deep Learning/ASG1/Saved Models/model_4.tf
150/150 [==============================] - 5453s 36s/step - loss: 2.2118 - acc: 0.

In [ ]:
model_4 = tuner.get_best_models(num_models = 1)[0]

In [ ]:
model_4.summary()

In [ ]:
model_4.save('/content/drive/My Drive/Ngee Ann Poly/Deep Learning/ASG1/Saved Models/model_4.tf')

## Step 3 – Evaluate the Developed Models using Testing Images

In [ ]:
model_4 = keras.models.load_model('/content/drive/My Drive/Ngee Ann Poly/Deep Learning/ASG1/Saved Models/model_4.tf')

evaluation_results = model_4.evaluate(test_generator, steps=len(test_generator))

print("Test Loss:", evaluation_results[0])
print("Test Accuracy:", evaluation_results[1])

In [ ]:
predictions = model_4.predict(test_generator, steps=len(test_generator), verbose=1)
predicted_labels = np.argmax(predictions, axis=1)

# Get the true labels from the test data generator
true_labels = test_generator.classes

# Create the classification report
class_report = classification_report(true_labels, predicted_labels, target_names=test_generator.class_indices.keys())
print("Classification Report:")
print(class_report)

In [ ]:
confusion_matrix = confusion_matrix(true_labels, predicted_labels)

plt.figure(figsize=(10, 10))
sns.heatmap(confusion_matrix, annot=True, fmt='d')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion matrix')
plt.show()

## Step 4  – Use the best model to make prediction

In [ ]:
# Load the food list (in alphabetical order)
with open('/content/drive/My Drive/Ngee Ann Poly/Deep Learning/ASG1/18.txt', 'r') as f: # the .txt file which contains a list of food assigned to you
    x = f.readlines()
food_list =[]
for item in x:
    food_list.append(item.strip('\n'))
food_list = sorted(food_list) # food_list needs to be sorted alphabetically before feed into prediction() function
print(food_list)

In [ ]:
# Define some related functions for image process and model prediction
from tensorflow.keras.preprocessing.image import load_img, img_to_array
def image_process(img):
    image = load_img(img, target_size =(img_size, img_size))
    image_array = img_to_array(image)/255
    return image_array

import pandas as pd
def prediction(model, img_array, items_l):
    prob = model.predict(img_array.reshape(1,img_size,img_size,3))
    pro_df = pd.DataFrame(prob, columns = items_l)
    result = items_l[np.argmax(prob)]
    return pro_df, result

In [ ]:
online_images_dir = '/content/drive/My Drive/Ngee Ann Poly/Deep Learning/ASG1/Online Images'
image_files = os.listdir(online_images_dir)

for image_file in image_files:
    img_path = os.path.join(online_images_dir, image_file)
    plt.imshow(plt.imread(img_path))
    plt.show()

    img_array = image_process(img_path)
    prob_df, result = prediction(model_4, img_array, food_list)

    print(f'\nImage: {image_file}')
    print('The prediction is:', result, '\n\n', prob_df)
    print('-' * 40)